<a href="https://colab.research.google.com/github/jmlDC/MediaBias-Thesis22-23/blob/Modeling/bi_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

reference
https://colab.research.google.com/github/keras-team/keras-io/blob/master/examples/nlp/ipynb/bidirectional_lstm_imdb.ipynb#scrollTo=6Rf3oHV3zOzD 

In [18]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from keras.preprocessing.text import Tokenizer


In [2]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 19.5 MB/s eta 0:00:00


In [3]:

max_features = 20000  # Only consider the top 20k words
maxlen = 200  # Only consider the first 200 words of each movie review


In [4]:
# Input for variable-length sequences of integers
inputs = keras.Input(shape=(None,), dtype="int32")
# Embed each integer in a 128-dimensional vector
x = layers.Embedding(max_features, 128)(inputs)
# Add 2 bidirectional LSTMs
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
# Add a classifier
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 128)         2560000   
                                                                 
 bidirectional (Bidirectiona  (None, None, 128)        98816     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 2,757,761
Trainable params: 2,757,761
Non-train

In [19]:
from google.colab import drive
drive.mount('/content/gdrive')

dir  = "/content/gdrive/MyDrive/THESIS-MS/Git-Thesis22-23/"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [21]:
from datasets import Dataset
import pandas as pd

df = pd.read_csv(f'{dir}Official/MFC_prepared.csv', usecols=["code_frames", "annotations"], header=0)
df.code_frames = df.code_frames.astype(int)

label_dict = {}
for x in range(15):
    label_dict[x+1] = x

label_dict
df['label'] = df.code_frames.replace(label_dict)
df

,code_frames,annotations,label
0,10,Immigrants without HOPE need help entering col...,9
1,5,"But in the eyes of the law, he is an illegal i...",4
2,15,"Reaction to Tancredo, Lamm as predicted",14
3,13,"That, said the congressman, is what always hap...",12
4,1,"$50,000 per entry",0
...,...,...,...
46794,11,Smoking is becoming a social taboo,10
46795,5,Nor does it aid lawyers seeking novel ways to...,4
46796,15,'Ashes to Ashes',14
46797,15,SMOKE SCREEN IS SEEN BEHIND THE SMOKELESS,14


In [14]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(df.annotations.values, 
                                                  df.label.values, 
                                                  test_size=0.15, 
                                                  random_state=42,
                                                  stratify=df.label.values
                                                )

In [20]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

In [26]:
def read_glove_vector(glove_vec):
  with open(glove_vec, 'r', encoding='UTF-8') as f:
    words = set()
    word_to_vec_map = {}
    for line in f:
      w_line = line.split()
      curr_word = w_line[0]
      word_to_vec_map[curr_word] = np.array(w_line[1:], dtype=np.float64)



  return word_to_vec_map

word_to_vec_map = read_glove_vector(f'{dir}glove.6B/glove.6B.100d.txt')



In [15]:
(x_train, y_train), (x_val, y_val) = keras.datasets.imdb.load_data(
    num_words=max_features
)
print(len(x_train), "Training sequences")
print(len(x_val), "Validation sequences")
x_train = keras.utils.pad_sequences(x_train, maxlen=maxlen)
x_val = keras.utils.pad_sequences(x_val, maxlen=maxlen)


25000 Training sequences
25000 Validation sequences


In [ ]:
model.compile("adam", "binary_crossentropy", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=32, epochs=2, validation_data=(x_val, y_val))


In [7]:
x_train, x_val

(array([[   5,   25,  100, ...,   19,  178,   32],
        [   0,    0,    0, ...,   16,  145,   95],
        [   0,    0,    0, ...,    7,  129,  113],
        ...,
        [   0,    0,    0, ...,    4, 3586,    2],
        [   0,    0,    0, ...,   12,    9,   23],
        [   0,    0,    0, ...,  204,  131,    9]], dtype=int32),
 array([[    0,     0,     0, ...,    14,     6,   717],
        [ 1987, 17975,    45, ...,   125,     4,  3077],
        [ 4468,   189,     4, ...,     9,    57,   975],
        ...,
        [    0,     0,     0, ...,    21,   846,  5518],
        [    0,     0,     0, ...,  2302,     7,   470],
        [    0,     0,     0, ...,    34,  2005,  2643]], dtype=int32))

In [16]:
X_train, X_val

(array(['ATTACKING A BASIC AMERICAN RIGHT',
        'How house voted on 7-day gun wait',
        ' to vacate the death sentence and order his sentence commuted to life in prison.',
        ..., 'Open-records chief slams Corbett over gay-marriage remark',
        'that is replaying the horrific details of the 2013 Boston Marathon bombing,',
        ' Based on a bill authored by Assemblyman Pete Grannis (D-Manhattan), it was expected to pass both the Assembly and Senate last night. Grannis predicted the federal government will follow suit.'],
       dtype=object),
 array(["The Kerrys and their cousin Thomas Cummins were visiting the bridge one night when they came upon Richardson and three of his friends. The Kerrys were raped at Richardson's suggestion, according to court documents, and both sisters and Cummins were forced to jump from the 70-foot-high bridge into the Mississippi River.",
        'After Conn. tragedy, some in Pa. look again at gun laws',
        "For the Families, the D